In [1]:
from transformers import pipeline



/Users/ameezingwongstein/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

/Users/ameezingwongstein/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB"\n\n"Somewhat important" said the man, adding he was in fact a "wishlist"'},
 {'generated_text': 'I HATE MY JOB!"\n\nThe question about why these parents are so proud of their children: A. Is everything better than ever in'},
 {'generated_text': 'I HATE MY JOB AND HATE THAT MAN," said the father, who said he had recently completed a six-week course for the SAT'}]

## Idea 1
Train text generator on Amazon bad reviews
